# Impulse-response functions based on the distributed lag models
#### Christopher Callahan
#### Christopher.W.Callahan.GR@dartmouth.edu

The idea here is that we want to force the DL coefficients to equal zero at some time so that the growth effects are explicitly not permanent -- IRF refers to the time structure of a dose-response function. 

#### Mechanics
Dependencies

In [2]:
import xarray as xr
import numpy as np
import sys
import os
import datetime
import pandas as pd

Data location

In [4]:
loc_regression = "../Data/RegressionResults/"
loc_out = "..Data/IRFs/"

#### Analysis

Initial stuff

In [5]:
nlag = 5 # when reading in DL model results
boot_type = "region"
lags = np.arange(0,nlag+1,1)

In [6]:
extr = "tx5d"
convergence_years = [3,np.nan,4]

vrs = [extr,"temperature","variability"]

for vv in np.arange(0,len(vrs),1):
    v = vrs[vv]
    convergence_year = convergence_years[vv]
    print(v)

    if v == extr:
        coefs_main = pd.read_csv(loc_regression+extr+"_coefs_bootstrap_"+boot_type+"_lag"+str(nlag)+".csv",index_col=0)
        coefs_interact = pd.read_csv(loc_regression+extr+"_coefs_interact_bootstrap_"+boot_type+"_lag"+str(nlag)+".csv",index_col=0)
    elif v == "variability":
        coefs_main = pd.read_csv(loc_regression+v+"_coefs_"+extr+"_model_bootstrap_"+boot_type+"_lag"+str(nlag)+".csv",index_col=0)
        coefs_interact = pd.read_csv(loc_regression+v+"_coefs_interact_"+extr+"_model_bootstrap_"+boot_type+"_lag"+str(nlag)+".csv",index_col=0)
    elif v == "temperature":
        coefs_main = pd.read_csv(loc_regression+v+"_coefs_"+extr+"_model_bootstrap_"+boot_type+"_lag"+str(nlag)+".csv",index_col=0)
        ## temperature only has a significant interaction with tx5d, not with itself (i.e., the t^2 term is not significant)
        coefs_interact = pd.read_csv(loc_regression+extr+"_coefs_interact_bootstrap_"+boot_type+"_lag"+str(nlag)+".csv",index_col=0)
    
    reg_boot = coefs_main.boot.values
    nboot = len(reg_boot)

    coefs_main_xr = xr.DataArray(coefs_main.iloc[:,1:].values,
                            coords=[reg_boot,lags],dims=["reg_boot","lag"])
    coefs_int_xr = xr.DataArray(coefs_interact.iloc[:,1:].values,
                            coords=[reg_boot,lags],dims=["reg_boot","lag"])
    
    
    # now construct IRF out to desired convergence year
    if ~np.isnan(convergence_year):
        
        lags2 = np.arange(0,convergence_year+1,1)
        coef_main_irf = xr.DataArray(np.zeros((nboot,convergence_year+1)),
                                    coords=[reg_boot,lags2],dims=["reg_boot","lag"])
        coef_int_irf = xr.DataArray(np.zeros((nboot,convergence_year+1)),
                                    coords=[reg_boot,lags2],dims=["reg_boot","lag"])

        # initial response
        coef_main_irf[:,0] = coefs_main_xr[:,0].values
        coef_int_irf[:,0] = coefs_int_xr[:,0].values

        # peak in desired year
        for x in np.arange(1,convergence_year,1):
            ls = list(np.arange(0,x+1,1))
            coef_main_irf[:,x] = coefs_main_xr.loc[:,ls].sum(dim="lag")
            coef_int_irf[:,x] = coefs_int_xr.loc[:,ls].sum(dim="lag")

        # last period cumulative response already zero (since we used np.zeros)

        # now differentiate to produce the actual coefficients that yield that cumulative response
        coef_main_irf_final = xr.concat([coef_main_irf.loc[:,0],coef_main_irf.diff("lag",1)],dim="lag")
        coef_int_irf_final = xr.concat([coef_int_irf.loc[:,0],coef_int_irf.diff("lag",1)],dim="lag")
        
    else:
        
        ## if it never converges to zero, the IRF coefficients are just
        ## the same as the DL coefficients

        coef_main_irf_final = coefs_main_xr*1.0
        coef_int_irf_final = coefs_int_xr*1.0
    
    # write out data
    irf_ds = xr.Dataset({"coef_main":(["reg_boot","lag"],coef_main_irf_final),
                         "coef_interact":(["reg_boot","lag"],coef_int_irf_final)},
                        coords={"reg_boot":(["reg_boot"],coef_main_irf_final.reg_boot),
                                "lag":(["lag"],coef_main_irf_final.lag)})

    irf_ds.attrs["creation_date"] = str(datetime.datetime.now())
    irf_ds.attrs["created_by"] = "Christopher Callahan, Christopher.W.Callahan.GR@dartmouth.edu"
    irf_ds.attrs["variable_description"] = "Impulse-response functions for each variable's effect on subnational economic growth"
    irf_ds.attrs["created_from"] = os.getcwd()+"/Calculate_IRFs.ipynb"
    irf_ds.attrs["dims"] = "reg_boot x lag"
    
    if v == extr:
        fname_out = loc_out+extr+"_impulse-response_coefficients_bootstrap_"+boot_type+"_lag"+str(nlag)+".nc"
    else:
        fname_out = loc_out+v+"_impulse-response_coefficients_"+extr+"_model_bootstrap_"+boot_type+"_lag"+str(nlag)+".nc"
    irf_ds.to_netcdf(fname_out,mode="w")
    print(fname_out,flush=True)

tx5d
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/Extremes_Economics/Data/IRFs/SciAdv_Revision1/tx5d_impulse-response_coefficients_bootstrap_region_lag5.nc
temperature
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/Extremes_Economics/Data/IRFs/SciAdv_Revision1/temperature_impulse-response_coefficients_tx5d_model_bootstrap_region_lag5.nc
variability
/dartfs-hpc/rc/lab/C/CMIG/ccallahan/Extremes_Economics/Data/IRFs/SciAdv_Revision1/variability_impulse-response_coefficients_tx5d_model_bootstrap_region_lag5.nc
